# Data Science / Machine Learning Meetup #1 Deep Learning Hands-on
# オルタナティブ・データと自然言語処理

## 手順
各問題は、実装する関数と、関数の実装方法に関する指示で構成されています。実装する必要がある関数の部分には# TODOコメントが付いています。

## 1. 環境準備

### パッケージのインストールとインポート

In [1]:
!pip3 install nltk
!pip3 install torch

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip3 install ipython-sql

You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
!pip3 install pyhive
!pip3 install sqlalchemy
!pip3 install thrift

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 61kB 6.6MB/s eta 0:00:01
  Stored in directory: /home/cdsw/.cache/pip/wheels/02/a2/46/689ccfcf40155c23edc7cdbd9de488611c8fdf49ff34b1706e
Successfully built thrift
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
!pip3 install sasl
!pip3 install thrift_sasl
# These are not used but it's easier than the server-side configuration mentioned below.
# https://github.com/apache/incubator-superset/issues/4782

In [ ]:
#$ beeline -u 'jdbc:hive2://10.0.0.55:10000' -f tables.hql

In [3]:
from pyhive import hive

In [4]:
import sqlalchemy
sqlalchemy.create_engine('hive://ip-10-0-0-55.ap-northeast-1.compute.internal:10000')
#sqlalchemy.create_engine('thrift://ip-10-0-0-55.ap-northeast-1.compute.internal:9083')

Engine(hive://ip-10-0-0-55.ap-northeast-1.compute.internal:10000)

In [5]:
%load_ext sql

**下記のセルの中を適切なHiveサーバのURLに置換してください。**

In [6]:
%sql hive://ip-10-0-0-55.ap-northeast-1.compute.internal:10000

'Connected: @None'

In [7]:
%%sql
show tables

 * hive://ip-10-0-0-55.ap-northeast-1.compute.internal:10000
Done.


tab_name
json_message
message_exploded
message_extracted
message_filtered
sentiment_data
twits


In [13]:
!echo $HADOOP_USER_NAME

user1


In [17]:
!pip3 install hive_service

  ERROR: Could not find a version that satisfies the requirement hive_service (from versions: none)
ERROR: No matching distribution found for hive_service
You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:

from pyhive import hive
from TCLIService.ttypes import TOperationState
cursor = hive.connect('ip-10-0-0-55.ap-northeast-1.compute.internal', port=10000).cursor()
cursor.execute('show tables', async=True)

status = cursor.poll().operationState
while status in (TOperationState.INITIALIZED_STATE, TOperationState.RUNNING_STATE):
    logs = cursor.fetch_logs()
    for message in logs:
        print(message)
    status = cursor.poll().operationState

print(cursor.fetchall())

[('json_message',), ('message_exploded',), ('message_extracted',), ('message_filtered',), ('sentiment_data',), ('twits',)]


In [15]:
%%sql
create `$HADOOP_USER_NAME` user1 IF NOT EXISTS

 * hive://ip-10-0-0-55.ap-northeast-1.compute.internal:10000
(pyhive.exc.OperationalError) TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=["*org.apache.hive.service.cli.HiveSQLException:Error while compiling statement: FAILED: ParseException line 1:7 cannot recognize input near 'create' '$HADOOP_USER_NAME' 'user1' in ddl statement:29:28", 'org.apache.hive.service.cli.operation.Operation:toSQLException:Operation.java:329', 'org.apache.hive.service.cli.operation.SQLOperation:prepare:SQLOperation.java:207', 'org.apache.hive.service.cli.operation.SQLOperation:runInternal:SQLOperation.java:290', 'org.apache.hive.service.cli.operation.Operation:run:Operation.java:260', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatementInternal:HiveSessionImpl.java:505', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatement:HiveSessionImpl.java:480', 'sun.reflect.NativeMethodAccessorImpl:invoke0:NativeMethodAccessorImpl.java:-2', 'sun.reflect.NativeMethodAcc

In [10]:
%%sql
use user1

 * hive://ip-10-0-0-55.ap-northeast-1.compute.internal:10000
Done.


[]

In [19]:
%%sql
show tables

 * hive://ip-10-0-0-55.ap-northeast-1.compute.internal:10000
Done.


tab_name


In [16]:
%%sql
select * from json_message limit 5

 * hive://ip-10-0-0-55.ap-northeast-1.compute.internal:10000
Done.


message
"{""message_body"":""It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle $CMCSA $VIAC $AAPL $AMZN $DIS "",""sentiment"":0}"
"{""message_body"":""$JD $AAPL"",""sentiment"":0}"
"{""message_body"":""$SPY $AAPL $CLVS $AMRN "",""sentiment"":0}"
"{""message_body"":""$AAPL Repo 8AM tomorrow"",""sentiment"":2}"
"{""message_body"":""$AAPL bought 320 puts at the hod should open nicely looking for 70-80%+ at open"",""sentiment"":-2}"


In [3]:
from pyhive import hive
from TCLIService.ttypes import TOperationState
cursor = hive.connect('ip-10-0-0-55.ap-northeast-1.compute.internal', port=10000).cursor()
cursor.execute('show tables', async=True)

status = cursor.poll().operationState
while status in (TOperationState.INITIALIZED_STATE, TOperationState.RUNNING_STATE):
    logs = cursor.fetch_logs()
    for message in logs:
        print(message)
    status = cursor.poll().operationState

print(cursor.fetchall())

[('json_message',), ('message_exploded',), ('message_extracted',), ('message_filtered',), ('sentiment_data',), ('twits',)]


In [3]:
import json
import os
import random
import re
import subprocess
import glob

import torch
import nltk
from torch import nn, optim
import torch.nn.functional as F

In [29]:
HDFS_PATH_DIR = '/tmp/'
HDFS_PATH_DIR = './'

args = ['hdfs', 'dfs', '-put', '', HDFS_PATH_DIR]

file_list = glob.glob("./lib/*")

#proc = subprocess.call(["hadoop", "fs", "-ls"])
#proc = subprocess.run(["hdfs", "dfs", "-ls"])
#print("ls stdout:", proc.stdout.decode("utf8"))

for file in file_list:
  try:
    #res = subprocess.check_call(args)
    args[3] = file
    print(args)

    proc = subprocess.run(args,stdout = subprocess.PIPE, stderr = subprocess.PIPE)
    print("stdout:", proc.stdout.decode("utf8"))
    
    #path = FILE_PATH + symbol + ".json"
    
    #with open(path, mode='w') as f:
    #  f.write(proc.stdout.decode("utf8"))
  
  except:
    import traceback
    traceback.print_exc()
    print("Error.")

['hdfs', 'dfs', '-put', './lib/json-1.3.7.3.jar', './']
stdout: 
['hdfs', 'dfs', '-put', './lib/README.md', './']
stdout: 
['hdfs', 'dfs', '-put', './lib/brickhouse-0.7.1-SNAPSHOT.jar', './']
stdout: 
['hdfs', 'dfs', '-put', './lib/json-serde-cdh5-shim-1.3.7.3.jar', './']
stdout: 
['hdfs', 'dfs', '-put', './lib/json-serde-1.3.7.3.jar', './']
stdout: 


## 2. Web Scraping

In [3]:
!mkdir ./data

In [52]:
import subprocess
from datetime import datetime

symbols = ['BBRY', 'AAPL', 'AMZN', 'BABA', 'YHOO', 'LQMT', 'FB', 'GOOG', 'BBBY', 'JNUG', 'SBUX', 'MU']

args = ['curl', '-X', 'GET', '']
URL = "https://api.stocktwits.com/api/2/streams/symbol/"

FILE_PATH = "./data/"

for i in range(10):
    start_datetime = datetime.now().strftime("%Y%m%d_%H%M")
    for symbol in symbols:
        try:
            args[3] = URL + symbol + ".json"
            print(args[3])
            proc = subprocess.run(args,stdout = subprocess.PIPE, stderr = subprocess.PIPE)

            path = FILE_PATH + symbol + "_" + start_datetime + ".json"
            print(path)
            with open(path, mode='w') as f:
                f.write(proc.stdout.decode("utf8"))
        except:
            import traceback
            traceback.print_exc()
            print("Error.")

https://api.stocktwits.com/api/2/streams/symbol/BBRY.json
./data/BBRY_20200123_0733.json
https://api.stocktwits.com/api/2/streams/symbol/AAPL.json
./data/AAPL_20200123_0733.json
https://api.stocktwits.com/api/2/streams/symbol/AMZN.json
./data/AMZN_20200123_0733.json
https://api.stocktwits.com/api/2/streams/symbol/BABA.json
./data/BABA_20200123_0733.json
https://api.stocktwits.com/api/2/streams/symbol/YHOO.json
./data/YHOO_20200123_0733.json
https://api.stocktwits.com/api/2/streams/symbol/LQMT.json
./data/LQMT_20200123_0733.json
https://api.stocktwits.com/api/2/streams/symbol/FB.json
./data/FB_20200123_0733.json
https://api.stocktwits.com/api/2/streams/symbol/GOOG.json
./data/GOOG_20200123_0733.json
https://api.stocktwits.com/api/2/streams/symbol/BBBY.json
./data/BBBY_20200123_0733.json
https://api.stocktwits.com/api/2/streams/symbol/JNUG.json
./data/JNUG_20200123_0733.json
https://api.stocktwits.com/api/2/streams/symbol/SBUX.json
./data/SBUX_20200123_0733.json
https://api.stocktwits.co

./data/JNUG_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/SBUX.json
./data/SBUX_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/MU.json
./data/MU_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/BBRY.json
./data/BBRY_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/AAPL.json
./data/AAPL_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/AMZN.json
./data/AMZN_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/BABA.json
./data/BABA_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/YHOO.json
./data/YHOO_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/LQMT.json
./data/LQMT_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/FB.json
./data/FB_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/GOOG.json
./data/GOOG_20200123_0734.json
https://api.stocktwits.com/api/2/streams/symbol/BBBY.json
./data/BBBY_20200123_0734.jso

## HDFSへファイルをコピー

In [53]:


#https://community.cloudera.com/t5/CDH-Manual-Installation/amp-quot-RuntimeException-core-site-xml-not-found-amp-quot/td-p/87590
# So, I assume that the problem comes from the code where HIVE_CONF_DIR is appended to HADOOP_CONF_DIR. 
#
#os.environ['HADOOP_CONF_DIR'] = "/opt/cloudera/parcels/CDH-6.1.0-1.cdh6.1.0.p0.770702/lib/spark/conf/yarn-conf"
#os.environ['HADOOP_CONF_DIR'] = "/etc/spark/conf/yarn-conf"

#os.environ['HADOOP_CONF_DIR'] = "/opt/cloudera/parcels/CDH-6.3.2-1.cdh6.3.2.p0.1605554/lib/spark/conf/yarn-conf"
os.environ['HADOOP_CONF_DIR'] = "/etc/spark/conf/yarn-conf"

#args = ['hdfs', 'dfs', '-ls', '/tmp/']
HDFS_PATH_DIR = '/tmp/twits/'

#HDFS_PATH_DIR = '/tmp/'

args = ['hdfs', 'dfs', '-put', '', HDFS_PATH_DIR]


try:
  args_mkdir = ['hdfs', 'dfs', '-mkdir', HDFS_PATH_DIR]
  proc = subprocess.run(args_mkdir,stdout = subprocess.PIPE, stderr = subprocess.PIPE)
  
except:
  import traceback
  traceback.print_exc()
  print("Error.")


file_list = glob.glob("./data/*")


for file in file_list:
    try:
        #res = subprocess.check_call(args)
        args[3] = file
        print(args)
        #subprocess.call(["hadoop", "fs", "-ls"])
        proc = subprocess.run(args,stdout = subprocess.PIPE, stderr = subprocess.PIPE)
        print(proc.stdout.decode("utf8"))

        #path = FILE_PATH + symbol + ".json"

        #with open(path, mode='w') as f:
        #  f.write(proc.stdout.decode("utf8"))
    except:
        import traceback
        traceback.print_exc()
        print("Error.")

['hdfs', 'dfs', '-put', './data/GOOG_20200123_0730.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/AAPL_20200123_0734.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/BABA_20200123_0717.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/MU_20200123_0732.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/BABA_20200123_0613.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/SBUX_20200123_0613.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/GOOG_20200123_0733.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/MU_20200123_0730.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/SBUX_20200123_0618.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/BBRY_20200123_0733.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/GOOG_20200123_0618.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/JNUG_20200123_0717.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/YHOO_20200123_0717.json', '/tmp/twits/']

['hdfs', 'dfs', '-put', './data/LQMT_20200

## 3. データ変換

In [10]:
from __future__ import print_function
import sys
from random import random
from operator import add
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("JsonGen")\
    .getOrCreate()
    
spark.sparkContext.setLogLevel("ERROR")

#json_list = spark.read.table("json_message")
json_list = spark.sql("select * from json_message")

#json_list.show(5)

path = "./output.json"
with open(path, mode='w') as f:
    f.write('{"data":[')
    bool_first_line = True
    for row in json_list.rdd.collect():
        if bool_first_line:
        bool_first_line = False
    else:
        f.write(",\n")
    print(row.message)
    #f.write(row.message.encode("utf-8"))
    f.write(row.message)
    
    f.write("]}")

{"message_body":"It’s ‘Star Trek’ vs. ‘Star Wars’ in European Streaming Battle  $CMCSA $VIAC $AAPL $AMZN $DIS ","sentiment":0}
{"message_body":"$JD $AAPL","sentiment":0}
{"message_body":"$SPY $AAPL $CLVS $AMRN ","sentiment":0}
{"message_body":"$AAPL Repo 8AM tomorrow","sentiment":2}
{"message_body":"$AAPL bought 320 puts at the hod should open nicely looking for 70-80%+ at open","sentiment":-2}
{"message_body":"$AAPL VIRUS GANG","sentiment":-2}
{"message_body":"$SPY think of the last person in china who touched your new iphone before you open the box","sentiment":-2}
{"message_body":"$SPY $AMD $FB $BABA $AAPL By Morning everyone will be like","sentiment":0}
{"message_body":"Apple Reports Earnings Next Week: Here’s How to Tell if Apple Will Meet Expectations $AAPL @apple #apple #earnings http://bit.ly/2RigUf2","sentiment":0}
{"message_body":"$AAPL $335 before ER","sentiment":2}
{"message_body":"$AAPL this trades in really tight, neat channels all year. if there is a correction maximum d

## 4. 感情分析
企業の価値を決定するときは、ニュースをフォローすることが重要です。たとえば、会社の製品チェーンにおける製品のリコールまたは自然災害。この情報を信号に変換できるようにしたいと考えています。現在、この仕事に最適なツールはニューラルネットワークです。

このプロジェクトでは、ソーシャルメディアサイトStockTwitsの投稿を使用します。StockTwitsのコミュニティは、投資家、トレーダー、起業家により利用されています。投稿された各メッセージはTwitと呼ばれます。これはTwitterのツイートによく似ています。感情のスコアを生成するこれらのtwitを中心にモデルを構築します。

多数のtwitsを収集し、それぞれの感情を手でラベル付けしました。センチメントの度合いを把握するために、非常にネガティブ、ネガティブ、ニュートラル、ポジティブ、非常にポジティブという5段階のスケールを使用します。各ツイットは、それぞれ非常に負から非常に正まで、1のステップで-2から2までラベル付けされます。このラベル付きデータを使用して、感情を自分でtwitsに割り当てることを学習する感情分析モデルを構築します。

最初にすべきことは、データをロードすることです。

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: センチメントスコアは、-2から2の範囲で1のステップで、0は中立です。


データがどのように見えるかを確認します。

In [4]:
#with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
with open('./twits_dumped.json', 'r') as f:
#with open('./output.json', 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [5]:
"""print out the number of twits"""

# TODO Implement 

print(len(twits['data']))

1548010


### Split Message Body and Sentiment Score

In [6]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## データの前処理
データを入手したら、テキストを前処理する必要があります。これらのtwitは、twit自体でリーダー$シンボルで示されるティッカーシンボルでフィルタリングすることにより収集されます。例えば、

{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}

ティッカーシンボルはセンチメントに関する情報を提供せず、すべてのツイットに含まれているため、削除する必要があります。このtwitには@googleユーザー名もあり、ここでもセンチメント情報は提供されないため、削除する必要があります。URLも表示されますhttp://t.co/sptHOAh8。これらも削除しましょう。

特定の単語やフレーズを削除する最も簡単な方法は、reモジュールを使用して正規表現を使用することです。スペースを使用して特定のパターンをサブアウトできます。

re.sub(pattern, ' ', text)
これにより、テキスト内のパターンが一致する場所でスペースが置換されます。後でテキストをトークン化するときに、それらのスペースで適切に分割します。

### Pre-Processing

In [7]:
nltk.download('wordnet')

def preprocess(message):
    """
    入力として文字列を受け取り、次の操作を実行する: 
        - 全てのアルファベットを小文字に変換
        - URLを削除
        - ティッカーシンボルを削除 
        - 句読点を削除
        - 文字列をスペースで分割しトークン化する
        - シングル・キャラクターのトークンを削除
    
    パラメータ
    ----------
        message : 前処理の対象テキストメッセージ
        
    戻り値
    -------
        tokens: 前処理後のトークン配列
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub("http(s)?://([\w\-]+\.)+[\w-]+(/[\w\- ./?%&=]*)?",' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub("\$[^ \t\n\r\f]+", ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub("@[^ \t\n\r\f]+", ' ', text)

    # Replace everything not a letter with a space
    text = re.sub("[^a-z]", ' ', text)
    
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w, pos='v') for w in tokens if len(w) > 1]
    
    return tokens

[nltk_data] Downloading package wordnet to /home/cdsw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Twitsメッセージ前処理
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

※この処理には、データのサイズに応じて多少時間がかかります。

In [8]:
# TODO Implement
#print(messages[:3])

tokenized = list(map(preprocess, messages))

print(tokenized[:3])
print(len(tokenized))

['$FITB great buy at 26.00...ill wait', '@StockTwits $MSFT', '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating']
[['great', 'buy', 'at', 'ill', 'wait'], [], ['staanalystalert', 'for', 'jefferies', 'maintain', 'with', 'rat', 'of', 'hold', 'set', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'be', 'buy']]
1548010


### Bag of Words

すべてのメッセージがトークン化されたので、語彙を作成し、コーパス全体で各単語が出現する頻度をカウントします。Counter関数を使用して、すべてのトークンをカウントアップします。
[`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter)

※この処理には、データのサイズに応じて多少時間がかかります。

In [9]:
from collections import Counter

#words = []
#for tokens in tokenized:
#    for token in tokens:
#        words.append(token)
out_list = tokenized
words = [element for in_list in out_list for element in in_list]

print(words[:13])
print(len(words))

"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

word_counts = Counter(words)
sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
vocab_to_int = {word:ii for ii, word in int_to_vocab.items()}

bow = []
for tokens in tokenized:
    bow.append([vocab_to_int[token] for token in tokens])

print(len(bow))
print(bow[:3])

# This BOW will not be used because it is not filtered to eliminate common words.

['great', 'buy', 'at', 'ill', 'wait', 'staanalystalert', 'for', 'jefferies', 'maintain', 'with', 'rat', 'of', 'hold']
17085084
1548010
[[173, 15, 12, 1603, 160], [], [562, 4, 1432, 965, 40, 170, 7, 77, 282, 182, 75, 12, 696, 236, 256, 590, 0, 15]]


### メッセージに現れる単語の頻度

ボキャブラリーを使用して、「the」、「and」、「it」などの最も一般的な単語の一部を削除します。
これらの単語は感情を特定するのに寄与せず、非常に一般的であるため、ニューラルネットワークの入力のノイズとなります。これらを除外することで、ネットワークの学習時間を短縮することができます。

また、ほんの数回しか使われていない、非常にまれな単語も削除します。ここでは、各単語のカウントをメッセージの数で除算する必要があります。次に、メッセージのごく一部にしか表示されない単語を削除します。

In [10]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

print("len(sorted_vocab):",len(sorted_vocab))
print("sorted_vocab - top:", sorted_vocab[:3])
print("sorted_vocab - least:", sorted_vocab[-15:])

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
total_count = len(words)
freqs = {word: count/total_count for word, count in word_counts.items()}

#print("freqs[supplication]:",freqs["supplication"] )
print("freqs[the]:",freqs["the"] )

"""
This was the post by Ricardo:

there's no exact value for low_cutoff and high_cutoff, 
however I'd recommend you to use 
a low_cutoff that's around 0.000002 and 0.000007 
(This depends on the values you get from your freqs calculations) and 
a high_cutofffrom 5 to 20 (this depends on the most_common values from the bow).
"""

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000002

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
"""
example_count = []
example_count.append(sorted_vocab.index("the"))
example_count.append(sorted_vocab.index("for"))
example_count.append(sorted_vocab.index("of"))
print(example_count)
high_cutoff = min(example_count)
"""
high_cutoff = 20
print("high_cutoff:",high_cutoff)
print("low_cutoff:",low_cutoff)

# The k most common words in the corpus. Use `high_cutoff` as the k.
#K_most_common = [word for word in sorted_vocab[:high_cutoff]]
K_most_common = sorted_vocab[:high_cutoff]

print("K_most_common:",K_most_common)


##  END of TODO Implement

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]

print("len(filtered_words):",len(filtered_words)) 

len(sorted_vocab): 98190
sorted_vocab - top: ['be', 'the', 'to']
sorted_vocab - least: ['valuetrader', 'supplication', 'biznatch', 'daysagosome', 'whewhat', 'impleme', 'deadbefore', 'spooktacular', 'billiionairre', 'englishmans', 'freakjng', 'shortieturds', 'meatportmusic', 'aposbeast', 'mammograms']
freqs[the]: 0.02332432196411794
high_cutoff: 20
low_cutoff: 2e-06
K_most_common: ['be', 'the', 'to', 'amp', 'for', 'utm', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will', 'up', 'buy', 'report', 'go', 'have', 'metric']
len(filtered_words): 9593


### フィルターされた単語を削除して語彙を更新する¶
ボキャブラリーに役立つ3つの変数を作成します。

In [11]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab =  {word:ii for ii, word in enumerate(filtered_words)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {ii:word for word, ii in vocab.items()}
# tokenized with the words not in `filtered_words` removed.

print("len(tokenized):", len(tokenized))

filtered = [[token for token in tokens if token in vocab] for tokens in tokenized]
print("len(filtered):", len(filtered))
print("tokenized[:1]", tokenized[:1])
print("filtered[:1]",filtered[:1])

len(tokenized): 1548010
len(filtered): 1548010
tokenized[:1] [['great', 'buy', 'at', 'ill', 'wait']]
filtered[:1] [['great', 'ill', 'wait']]


### クラスのバランス
最後の前処理ステップをいくつか行いましょう。twitのラベル付けを見ると、twitの50％がニュートラルであることがわかります。これは、毎回0を推測するだけで、ネットワークの精度が50％になることを意味します。ネットワークが適切に学習できるように、クラスのバランスを取る必要があります。つまり、それぞれのセンチメントスコアがデータにほぼ同じ頻度で表示されることを確認します。

ここでできることは、それぞれの例に目を通し、中立的な感情を持つtwitsをランダムにドロップすることです。50％のニュートラルから20％のニュートラルtwitを取得したい場合、これらのtwitをドロップする確率はどうなりますか？この機会に、長さ0のメッセージを削除する必要もあります。

In [12]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [13]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1945424732105255

Finally let's convert our tokens into integer ids which we can pass to the network.

In [14]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [29]:
import pickle
#from singer import Singer

#singer = Singer('Shanranran')

with open('vocab.pickle', 'wb') as f:
    pickle.dump(vocab, f)

## ニューラルネットワーク¶
これでボキャブラリーができたので、トークンをIDに変換し、それをネットワークに渡すことができます。ネットワークを定義します

下記は、ネットワークの概要です：

#### Embed -> RNN -> Dense -> Softmax
### Text classifier (テキスト分類器)実装
テキスト分類器を作成する前に、「RNNを使用したセンチメント分析」演習で作成した他のネットワーク（ここでは「SentimentRNN」と呼ばれるネットワーク、ここでは「TextClassifer」と呼びます）を覚えている場合、3つの主要な部分で構成されています：: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

このネットワークは、forwardパスで期待して構築したネットワークに非常に似ています 。シグモイドの代わりにsoftmaxを使用します。シグモイドを使用しないのは、NNの出力がバイナリではないためです。このネットワークでは、センチメントスコアには5つの結果があります。最も高い確率の結果を探しているため、softmaxの方が適しています。

In [15]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(self.embed_size, self.lstm_size, self.lstm_layers, dropout=self.dropout)
        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(lstm_size, output_size)
        
        self.softmax = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.lstm_layers, batch_size,self.lstm_size).zero_(),
                         weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        batch_size = nn_input.size(0)
        
        embeds = self.embedding(nn_input)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        
        #lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)    
        """
        remember here you do not have batch_first=True, 
        so accordingly shape your input. 
        Moreover, since now input is seq_length x batch you just need to transform lstm_out = lstm_out[-1,:,:].
        you don't have to use batch_first=True in this case, 
        nor reshape the outputs with .view just transform your lstm_out as advised and you should be good to go.
        """
        lstm_out = lstm_out[-1,:,:]
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        logps = self.softmax(out)
        
        
        return logps, hidden_state

### View Model

In [16]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.6426, -1.8112, -1.2617, -1.7752, -1.6584],
        [-1.6734, -1.8336, -1.2340, -1.7829, -1.6436],
        [-1.6663, -1.7982, -1.2406, -1.7910, -1.6636],
        [-1.6416, -1.8462, -1.2322, -1.8082, -1.6456]],
       grad_fn=<LogSoftmaxBackward>)


## トレーニング
### DataLoaderとバッチ処理
ここで、データをループするために使用できるジェネレーターを構築する必要があります。シーケンスをバッチとして渡すことができれば、より効率的です。入力テンソルは次のようになり(sequence_length, batch_size)ます。したがって、シーケンスが40トークンで、25シーケンスを渡す場合、入力サイズはになり(40, 25)ます。

シーケンスの長さを40に設定した場合、40トークンより多いまたは少ないメッセージをどう処理しますか？40トークン未満のメッセージの場合、空のスポットにゼロを埋め込みます。データを処理する前にRNNが何も開始しないように、必ずパッドを残しておく必要があります。メッセージに20個のトークンがある場合、40個の長いシーケンスの最初の20個のスポットは0になります。メッセージに40個を超えるトークンがある場合、最初の40個のトークンを保持します。

In [17]:
#def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
def dataloader(messages, labels, sequence_length=20, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [18]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 

split_frac = 0.98 # for small data
#split_frac = 0.8 # for big data

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(token_ids)*split_frac)
train_features, remaining_features = token_ids[:split_idx], token_ids[split_idx:]
train_labels, remaining_labels = sentiments[:split_idx], sentiments[split_idx:]

test_idx = int(len(remaining_features)*0.5)
valid_features, test_features = remaining_features[:test_idx], remaining_features[test_idx:]
valid_labels, test_labels = remaining_labels[:test_idx], remaining_labels[test_idx:]

In [19]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(9594, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (softmax): LogSoftmax()
)

### トレーニング実施

※この処理には、データのサイズに応じて、十分な時間が必要です。

GPUを備えた環境で実行する場合、ターミナルで以下のコマンドを実行することで、GPUが利用されていることを確認することができます（ GPU実行中、コマンド実行により表示されるテーブルの右上のVolatile GPU-Utilのパーセンテージ値が増えます）
```
$ watch nvidia-smi
```

In [22]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""
import numpy as np

epochs = 4 #pass
batch_size =  64#pass
batch_size =  512#pass
learning_rate = 0.001 #pass

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

val_losses = []
accuracy = []

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        hidden = model.init_hidden(labels.shape[0]) #pass
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        hidden = tuple([each.data for each in hidden])
        model.zero_grad()
        output, hidden = model(text_batch, hidden)
        loss = criterion(output.squeeze(), labels)
        loss.backward()
        clip = 5
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        # Accumulate loss
        val_losses.append(loss.item())
        
        correct_count = 0.0
        if steps % print_every == 0:
            model.eval()
            
            # Calculate accuracy
            ps = torch.exp(output)
            top_p, top_class = ps.topk(1, dim=1)
            #?top_class = top_class.to(device)
            #?labels = labels.to(device)

            correct_count += torch.sum(top_class.squeeze()== labels)
            accuracy.append(100*correct_count/len(labels))
            
            # TODO Implement: Print metrics
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                 "Step: {}...".format(steps),
                 "Loss: {:.6f}...".format(loss.item()),
                 "Val Loss: {:.6f}".format(np.mean(val_losses)),
                 "Collect Count: {}".format(correct_count),
                 "Accuracy: {:.2f}".format((100*correct_count/len(labels))),
                 # AttributeError: 'torch.dtype' object has no attribute 'type'
                 #"Accuracy Avg: {:.2f}".format(np.mean(accuracy))
                 )
            
            model.train()

Starting epoch 1
Epoch: 1/4... Step: 100... Loss: 0.880020... Val Loss: 1.001440 Collect Count: 330.0 Accuracy: 64.45
Epoch: 1/4... Step: 200... Loss: 0.959440... Val Loss: 0.968005 Collect Count: 319.0 Accuracy: 62.30
Epoch: 1/4... Step: 300... Loss: 0.811821... Val Loss: 0.947690 Collect Count: 348.0 Accuracy: 67.97
Epoch: 1/4... Step: 400... Loss: 0.797742... Val Loss: 0.931062 Collect Count: 347.0 Accuracy: 67.77
Epoch: 1/4... Step: 500... Loss: 0.829777... Val Loss: 0.918691 Collect Count: 345.0 Accuracy: 67.38
Epoch: 1/4... Step: 600... Loss: 0.844466... Val Loss: 0.907893 Collect Count: 340.0 Accuracy: 66.41
Epoch: 1/4... Step: 700... Loss: 0.830899... Val Loss: 0.899086 Collect Count: 356.0 Accuracy: 69.53
Epoch: 1/4... Step: 800... Loss: 0.818034... Val Loss: 0.890440 Collect Count: 342.0 Accuracy: 66.80
Epoch: 1/4... Step: 900... Loss: 0.818061... Val Loss: 0.883944 Collect Count: 352.0 Accuracy: 68.75
Epoch: 1/4... Step: 1000... Loss: 0.801603... Val Loss: 0.878443 Collect C

In [24]:
torch.save({'state_dict': model.state_dict()}, 'checkpoint.pth.tar')

## 予測（Prediction）関数の作成
### Prediction 
訓練されたモデルを手に入れたので、新しいツイットでそれを試して、それが適切に機能するかどうか確かめてください。新しいテキストについては、ネットワークに渡す前に最初に前処理する必要があることに注意してください。predictメッセージから予測ベクトルを生成する関数を実装します。

In [31]:
import glob
print(glob.glob("/home/cdsw/*"))

import pickle
import re
import nltk
import numpy as np

nltk.download('wordnet')

import torch

#from sentiment import TextClassifier

import os
import sys
cur_dir = os.path.dirname(os.path.abspath('__file__'))
print(cur_dir)
sys.path.append(cur_dir)

vocab_filename = 'vocab.pickle'
vocab_path = cur_dir + "/" + vocab_filename
vocab_l = pickle.load(open(vocab_path, 'rb'))

#model_path = cur_dir + "/" + "model.torch"
#model_l = torch.load(model_path, map_location='cpu')

model_l = TextClassifier(len(vocab_l)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
checkpoint = torch.load('./checkpoint.pth.tar')
model_l.load_state_dict(checkpoint['state_dict'])


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub("http(s)?://([\w\-]+\.)+[\w-]+(/[\w\- ./?%&=]*)?",' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub("\$[^ \t\n\r\f]+", ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub("@[^ \t\n\r\f]+", ' ', text)

    # Replace everything not a letter with a space
    text = re.sub("[^a-z]", ' ', text)
    
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w, pos='v') for w in tokens if len(w) > 1]
    
    return tokens


def predict_func(text, model, vocab):
    """ 
    Make a prediction on a single sentence.
    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.
    Returns
    -------
        pred : Prediction vector
    """

    tokens = preprocess(text)    

    # Filter non-vocab words
    tokens = [token for token in tokens if token in vocab] #pass
    # Convert words to ids
    tokens = [vocab[token] for token in tokens] #pass

    if len(tokens) == 0:
      raise UnknownWordsError

    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.LongTensor(tokens).view(-1, 1)))

    # Get the NN output       
    batch_size = 1
    hidden = model.init_hidden(batch_size) #pass
    
    logps, _ = model(text_input, hidden) #pass
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.round(logps.squeeze())#pass
    pred = torch.exp(logps) 
    
    return pred





def predict_api(args):
  text = args.get('text')
  try:
    result = predict_func(text, model_l, vocab_l)
    return result.detach().numpy()[0]
  except UnknownWordsError:
    return [0,0,1,0,0]
    

#args = {"text": "Google is working on self driving cars, I'm bullish on $goog"}
#args = {"text": "I'm bullish on $goog"}
args = {"text": "I'll strongly recommend to buy on $goog"}
#args = {"text": "elyoq baoq pquq $goog"}
result = predict_api(args)
print(result)

['/home/cdsw/checkpoint.pth.tar', '/home/cdsw/nlp_handson.ipynb', '/home/cdsw/twits_dumped.json', '/home/cdsw/test.py', '/home/cdsw/data', '/home/cdsw/lib', '/home/cdsw/nlp_solution.ipynb', '/home/cdsw/README.md', '/home/cdsw/model.torch', '/home/cdsw/nltk_data', '/home/cdsw/tables.hql', '/home/cdsw/ticker.txt', '/home/cdsw/vocab.pickle', '/home/cdsw/init.sh', '/home/cdsw/output.json']
/home/cdsw
[ 0.00204649  0.02024106  0.08254681  0.09021453  0.80495113]


[nltk_data] Downloading package wordnet to /home/cdsw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """
    
    # TODO Implement
    tokens = preprocess(text)

    # Filter non-vocab words
    tokens = [token for token in tokens if token in vocab] #pass
    # Convert words to ids
    tokens = [vocab[token] for token in tokens] #pass

    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.LongTensor(tokens).view(-1, 1)))

    # Get the NN output       
    batch_size = 1
    hidden = model.init_hidden(batch_size) #pass
    
    logps, _ = model(text_input, hidden) #pass
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.round(logps.squeeze())#pass
    pred = torch.exp(logps) 
    
    return pred

In [45]:
text = "Good good good wonderful"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[0.1989, 0.1649, 0.1976, 0.1587, 0.2799]], grad_fn=<ExpBackward>)

In [46]:
text = "Bad bad bad worst"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[0.1995, 0.1990, 0.2000, 0.1916, 0.2100]], grad_fn=<ExpBackward>)

In [47]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[0.2138, 0.1600, 0.2078, 0.1529, 0.2655]], grad_fn=<ExpBackward>)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

#### What is the prediction of the model?
The prediction to the text above is positive as the highest value is positive in the probability list - very negative, negative, neutral, positive, very positive.
#### What is the uncertainty of the prediction?
When considering the sum of the rest values except for the highest class, the uncertainty of the prediction is low and when taking into account the both positive and very positive, the uncertainty is very low. So, the prediction seems appropriate.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [24]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [25]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [26]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [27]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[ 0.1006,  0.1506,  0.2158,  0.2898,  0.2432]]),
 'timestamp': '2018-11-01T00:00:18Z'}

In [34]:
!pip3 freeze > requirements.txt

That's it. You have successfully built a model for sentiment analysis! 